# 统计`GSE92742_Broad_LINCS_sig_info.txt`中的元数据信息
| 列名              | 含义                                         | 示例                                      |
|------------------|--------------------------------------------|-----------------------------------------|
| sig_id           | 基因表达调控的唯一标识符，包含实验编号、细胞系、处理时间等信息 | AML001_CD34_24H:A05                       |
| pert_id          | 抑制剂的唯一标识符，用于标识化合物或处理类型    | DMSO 表示溶剂对照；BRD-A03772856 是具体化合物的ID。 |
| pert_iname       | 抑制剂的名称                                 | DMSO（对照）；trichostatin-a（具体药物）  |
| pert_type        | 抑制剂的类型                                 | ctl_vehicle（溶剂对照），trt_cp（化合物处理）  |
| cell_id          | 实验使用的细胞系                             | CD34: 表示实验细胞系为CD34。             |
| pert_dose        | 抑制剂的剂量                                 | 0.1, 单位后续列 pert_dose_unit 指定。      |
| pert_dose_unit   | 抑制剂剂量的单位                             | %, μM（微摩尔）                         |
| pert_idose       | 抑制剂量的解释性表示                         | 0.1%, 10 µM                             |
| pert_time        | 抑制剂处理的处理时间                         | 24h                                      |
| pert_time_unit   | 抑制剂处理时间的单位                         | h（小时）                               |
| pert_itime       | 抑制剂处理时间的解释性表示                   | 24 h                                    |
| distil_id        | 表示单个实验的重复样本，多个实验可能合并     | AML001_CD34_24H_X1_F1B10:J04            |

这行数据描述的是一个实验，实验编号为 AML001，使用 **CD34** 细胞系，处理时间为 **24小时**。实验中使用的扰动剂是 **trichostatin-a**，其 **pert_id** 为 BRD-A19037878，剂量为 1.11111 µM，处理持续时间为 **24小时**。该实验包含两个重复样本，分别位于 AML001_CD34_24H_X1_F1B10:F05 和 AML001_CD34_24H_X3_F1B10:F05。 

sig_id列名格式：`<实验编号>_<细胞系>_<时间>:<化合物>:<剂量>`

In [ ]:
"GSE92742_Broad_LINCS_sig_info.txt"
import pandas as pd

#加载sig_info文件
sig_info_path ="/mnt/d/Research/PHD/DLEPS/data/GSE92742/GSE92742_Broad_LINCS_sig_info.txt"
sig_info = pd.read_csv(sig_info_path, sep="\t", dtype=str)

#查看文件基本信息
unique_sig_ids = sig_info['sig_id'].nunique()
unique_pert_ids = sig_info['pert_id'].nunique()
unique_pert_iname = sig_info['pert_iname'].nunique()


# 输出结果
# print(sig_info)
print(f"不重复的实验记录数据(sig_id)数量: {unique_sig_ids}")
print(f"不重复的化合物标识号(pert_id)数量: {unique_pert_ids}")
print(f"不重复的化合物名称(pert_iname)数量: {unique_pert_iname}")



# 统计`GSE92742_Broad_LINCS_sig_info_final.txt`匹配SMILES后的数据信息

In [ ]:
"GSE92742_Broad_LINCS_sig_info_final.txt"
import pandas as pd

#加载sig_info文件
final_smiles_path ="/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_with_smiles.txt"
final_smiles = pd.read_csv(final_smiles_path, sep="\t", dtype=str)

#查看文件基本信息
final_unique_sig_ids = final_smiles['sig_id'].nunique()
final_unique_pert_ids = final_smiles['pert_id'].nunique()
final_unique_pert_iname = final_smiles['pert_iname'].nunique()
final_unique_canonical_smiles = final_smiles['canonical_smiles'].nunique()

# 输出结果
# print(final_smiles)
print(f"不重复的实验记录数据(sig_id)数量: {final_unique_sig_ids}")
print(f"不重复的化合物标识号(pert_id)数量: {final_unique_pert_ids}")
print(f"不重复的化合物名称(pert_iname)数量: {final_unique_pert_iname}")
print(f"不重复的SMILES数量: {final_unique_canonical_smiles}")

In [ ]:
import pandas as pd

# 读取文件
file_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_final.txt'
df = pd.read_csv(file_path, sep='\t')

# 计算不重复的 pert_id 的数量
unique_pert_ids = df['pert_id'].nunique()

# 输出结果
print(f"不重复的 pert_id 数量: {unique_pert_ids}")

# 查找 canonical_smiles 相同但 pert_id 不同的数据
duplicated_smiles = df[df.duplicated(subset=['canonical_smiles'], keep=False)]

# 按照 canonical_smiles 分组，检查是否有 pert_id 不同的情况
result = []
for smile, group in duplicated_smiles.groupby('canonical_smiles'):
    if group['pert_id'].nunique() > 1:
        result.append(group)

# 将结果合并并保存为 CSV 文件
if result:
    result_df = pd.concat(result)
    result_df.to_csv('/mnt/d/Research/PHD/DLEPS/results/duplicated_smiles_diff_pertid.csv', index=False)
    print("数据已保存到文件: duplicated_smiles_diff_pertid.csv")
else:
    print("没有找到具有相同 canonical_smiles 但不同 pert_id 的数据。")


# GPU加速测试

In [1]:
import tensorflow as tf
import time

def check_gpu():
    # 检查是否有GPU可用
    if tf.test.is_gpu_available():
        # 获取当前使用的GPU设备
        gpu_devices = tf.config.experimental.list_physical_devices('GPU')
        if gpu_devices:
            print("GPU设备可用！")
            for device in gpu_devices:
                print(f"GPU设备：{device.name}")
            run_gpu_computation()  # 进行GPU计算
        else:
            print("TensorFlow未能识别到GPU设备。")
    else:
        print("当前环境下没有GPU可用。")

def run_gpu_computation():
    # 创建一个简单的计算，执行矩阵乘法
    with tf.device('/GPU:0'):  # 确保使用第一个GPU
        print("开始在GPU上进行计算...")
        
        # 定义两个大矩阵进行乘法运算
        matrix_size = 1024  # 可以根据你的硬件调整大小
        A = tf.random.normal([matrix_size, matrix_size])
        B = tf.random.normal([matrix_size, matrix_size])
        
        start_time = time.time()
        C = tf.matmul(A, B)  # 矩阵乘法
        end_time = time.time()
        
        print("计算完成！")
        print(f"矩阵乘法计算耗时：{end_time - start_time}秒")
        
        # 如果需要查看GPU的详细信息，可以查看GPU利用率（需要nvidia-smi）
        # 可以通过外部工具如nvidia-smi来监控GPU使用情况
        print("如果要查看详细GPU使用情况，请在命令行中运行 `nvidia-smi`")

if __name__ == "__main__":
    check_gpu()


2024-12-12 13:58:23.829936: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2024-12-12 13:58:23.836419: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3187200000 Hz
2024-12-12 13:58:23.840163: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5569a2960a50 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-12-12 13:58:23.840190: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-12-12 13:58:23.841715: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1


GPU设备可用！
GPU设备：/physical_device:GPU:0
开始在GPU上进行计算...
计算完成！
矩阵乘法计算耗时：0.00019502639770507812秒
如果要查看详细GPU使用情况，请在命令行中运行 `nvidia-smi`


2024-12-12 13:58:24.168189: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-12 13:58:24.168282: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5569a2856d60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-12 13:58:24.168295: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-12-12 13:58:24.170136: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-12 13:58:24.170166: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 4090 major: 8 minor: 9 memoryClockRate(GHz): 2.52
pciBusID: